# DDPG, PPO, SAC under Partial Observability

In [1]:
import os
import cv2
import numpy as np
from utils import SparseDepth
from vision import Vision
from agentKinematics import RoboticAssistant

## Environment Setup

In [2]:
class IndoorDeepRL:
    def __init__(self, map_path="complex.png"):
        self.terra = cv2.flip(cv2.imread(map_path), 0)
        self.terra[self.terra > 128] = 255
        self.terra[self.terra <= 128] = 0
        self.m = np.asarray(self.terra)
        self.m = cv2.cvtColor(self.m, cv2.COLOR_RGB2GRAY)
        self.m = self.m.astype(float) / 255.
        self.terra = self.terra.astype(float) / 255.
        self.lmodel = Vision(self.m)

    def createInstance(self):
        self.robot = RoboticAssistant(d=5, wu=9, wv=4, car_w=9, car_f=7, car_r=10, dt=0.1)
        self.robot.x, self.robot.y = self.random_start_travesable()
        self.robot.theta = 360 * np.random.random()
        self.pos = (self.robot.x, self.robot.y, self.robot.theta)

        self.target = self.random_start_travesable()
        self.target_euclidian = np.sqrt((self.robot.x - self.target[0]) ** 2 + (self.robot.y - self.target[1]) ** 2)
        target_angle = np.arctan2(self.target[1] - self.robot.y, self.target[0] - self.robot.x) - np.deg2rad(self.robot.theta)
        target_distance = [self.target_euclidian * np.cos(target_angle), self.target_euclidian * np.sin(target_angle)]

        self.sdata, self.plist = self.lmodel.measure_depth(self.pos)
        state = self.existance(self.sdata, target_distance)
        return state

    def step(self, action):
        self.robot.control((action[0] + 1) / 2 * self.robot.v_interval, action[1] * self.robot.w_interval)
        self.robot.update()

        e1,e2,e3,e4 = self.robot.dimensions
        ee1 = SparseDepth(e1[0], e2[0], e1[1], e2[1])
        ee2 = SparseDepth(e1[0], e3[0], e1[1], e3[1])
        ee3 = SparseDepth(e3[0], e4[0], e3[1], e4[1])
        ee4 = SparseDepth(e4[0], e2[0], e4[1], e2[1])
        check = ee1+ee2+ee3+ee4
        
        collision = False
        for points in check:
            if self.m[int(points[1]),int(points[0])]<0.5:
                collision = True
                self.robot.redo()
                self.robot.velocity = -0.5 * self.robot.velocity
                break
        
        self.pos = (self.robot.x, self.robot.y, self.robot.theta)
        self.sdata, self.plist = self.lmodel.measure_depth(self.pos)

        action_r = 0.05 if action[0] < -0.5 else 0
        
        curr_target_dist = np.sqrt((self.robot.x - self.target[0]) ** 2 + (self.robot.y - self.target[1]) ** 2)
        distance_reward = self.target_euclidian - curr_target_dist
   
        s_orien = np.rad2deg(np.arctan2(self.target[1] - self.robot.y, self.target[0] - self.robot.x))
        orientation_error = (s_orien - self.robot.theta) % 360
        if orientation_error > 180:
            orientation_error = 360 - orientation_error
        orientation_reward = np.deg2rad(orientation_error)
  
        reward = distance_reward - orientation_reward - 0.6 * action_r
        
        terminated = False
        
        if curr_target_dist < 20:
            reward = 20
            terminated = True
        if collision:
            reward = -15
            terminated = True
        
        self.target_euclidian = curr_target_dist
        target_angle = np.arctan2(self.target[1] - self.robot.y, self.target[0] - self.robot.x) - np.deg2rad(self.robot.theta)
        target_distance = [self.target_euclidian * np.cos(target_angle), self.target_euclidian * np.sin(target_angle)]
        state_next = self.existance(self.sdata, target_distance)

        return state_next, reward, terminated

    def render(self, gui=True):
        experiment_space = self.terra.copy()
        for pts in self.plist:
            cv2.line(
                experiment_space,
                (int(1*self.pos[0]), int(1*self.pos[1])), 
                (int(1*pts[0]), int(1*pts[1])),
                (0.0,1.0,0.0), 1)

        cv2.circle(experiment_space, (int(1*self.target[0]), int(1*self.target[1])), 10, (1.0,0.5,0.7), 3)
        experiment_space = self.robot.render(experiment_space)
        experiment_space = cv2.flip(experiment_space,0)
        if gui:
            cv2.imshow("Mapless Navigation",experiment_space)
            k = cv2.waitKey(1)

        return experiment_space.copy()
    
    def random_start_travesable(self):
        height, width = self.m.shape[0], self.m.shape[1]
        tx = np.random.randint(0,width)
        ty = np.random.randint(0,height)

        kernel = np.ones((10,10),np.uint8)  
        m_dilate = 1-cv2.dilate(1-self.m, kernel, iterations=3)
        while(m_dilate[ty, tx] < 0.5):
            tx = np.random.randint(0,width)
            ty = np.random.randint(0,height)
        return tx, ty
    
    def existance(self, sensor, target):
        si = [s/200 for s in sensor]
        ti = [t/500 for t in target]
        return si + ti

## Trainning and Path configurations

In [3]:
training = True
render = True
load_model = False
terrain = "map.png"
gif_path = "performance/"
model_path = "models/"
if not os.path.exists(model_path):
    os.makedirs(model_path)

## Episode Visualization 

In [4]:
from PIL import Image

def performance_measure(episode, agent, s_count, max_rate):
     if episode>0 and episode%50==0:
        s_rate = s_count / 50
        if s_rate >= max_rate:
            max_rate = s_rate
            if training:
                print("Save model to " + model_path)
                agent.save_load_model("save", model_path)
        print("Success Rate (current/max):", s_rate, "/", max_rate)
     return max_rate

def visualize(agent, total_eps=2, message=False, render=False, map_path="large.png", gif_path="performance/", gif_name="test.gif"):
    if not os.path.exists(gif_path):
        os.makedirs(gif_path)

    images = []

    mother_nature = IndoorDeepRL(map_path=terrain)
    for eps in range(total_eps):
        step = 0
        max_success_rate = 0
        success_count = 0

        state = mother_nature.createInstance()
        r_eps = []
        acc_reward = 0.

        while(True):
            action = agent.choose_action(state, eval=True)
            state_next, reward, terminated = mother_nature.step(action)
            displayed = mother_nature.render(gui=render)
            im_pil = Image.fromarray(cv2.cvtColor(np.uint8(displayed*255),cv2.COLOR_BGR2RGB))
            images.append(im_pil)
            r_eps.append(reward)
            acc_reward += reward
            
            if message:
                print('\rEps: {:2d}| Step: {:4d} | action:{:+.2f}| R:{:+.2f}| Reps:{:.2f}  '\
                        .format(eps, step, action[0], reward, acc_reward), end='')
            
            state = state_next.copy()
            step += 1
            
            if terminated or step>200:
                if message:
                    print()
                break

    print("Create GIF ...")
    if gif_path is not None:
        images[0].save(gif_path+gif_name, save_all=True, append_images=images[1:], optimize=True, duration=40, loop=0)

## DDPG Agent training

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import ddpg

batch_size = 64
LOG_SIG_MAX = 2
LOG_SIG_MIN = -20
epsilon = 1e-6

class PolicyNet(nn.Module):
    def __init__(self):
        super(PolicyNet, self).__init__()
        self.layer1 = nn.Linear(23, 512)
        self.layer2 = nn.Linear(512, 512)
        self.layer3 = nn.Linear(512, 512)
        self.layer4 = nn.Linear(512, 2)

    def forward(self, s):
        hidden_layer_1 = F.relu(self.layer1(s))
        hidden_layer_2 = F.relu(self.layer2(hidden_layer_1))
        hidden_layer_3 = F.relu(self.layer3(hidden_layer_2))
        return torch.tanh(self.layer4(hidden_layer_3)) # one mu

class QNet(nn.Module):
    def __init__(self):
        super(QNet, self).__init__()
        self.layer1 = nn.Linear(23, 512)
        self.layer2 = nn.Linear(512+2, 512)
        self.layer3 = nn.Linear(512, 512)
        self.layer4 = nn.Linear(512, 1)
    
    def forward(self, s, a):
        hidden_layer_1 = F.relu(self.layer1(s))
        hidden_layer_1_a = torch.cat((hidden_layer_1, a), 1)
        hidden_layer_2 = F.relu(self.layer2(hidden_layer_1_a))
        hidden_layer_3 = F.relu(self.layer3(hidden_layer_2))
        return self.layer4(hidden_layer_3)


agent_mind_ddpg = ddpg.DDPG(base_net = [PolicyNet, QNet], b_size = batch_size)

In [6]:
if load_model:
    print("Load model ...", model_path)
    agent_mind_ddpg.save_load_model("load", model_path)

mother_nature = IndoorDeepRL(map_path=terrain)
total_steps = 0
max_success_rate = 0
success_count = 0
print("eps, step, total_steps, action[0], reward, loss_a, loss_c, agent_mind_ddpg.epsilon, acc_reward/step")

for eps in range(4500):
    state = mother_nature.createInstance()
    step = 0
    loss_a = loss_c = 0.
    acc_reward = 0.
    while True:
        if training:
            action = agent_mind_ddpg.choose_action(state, eval=False)
        else:
            action = agent_mind_ddpg.choose_action(state, eval=True)
        
        state_next, reward, terminated = mother_nature.step(action)
        end = 0 if terminated else 1
        agent_mind_ddpg.store_transition(state, action, reward, state_next, end)
        
        displayed = mother_nature.render(gui=render)

        loss_a = loss_c = 0.
        if total_steps > batch_size and training:
            loss_a, loss_c = agent_mind_ddpg.learn()
        step += 1
        total_steps += 1

        acc_reward += reward
        print('\r{:3d} ; {:4d}; {:6d}; {:+.2f}; {:+.2f}; {:+.2f}; {:+.2f}; {:.3f}; {:.2f}'
              .format(eps, step, total_steps, action[0], reward, loss_a, loss_c, agent_mind_ddpg._e, acc_reward/step), end='')
        state = state_next.copy()
        
        if terminated or step>200:
            if reward > 5:
                success_count += 1
            print()
            break

    max_success_rate = performance_measure(eps, agent_mind_ddpg, success_count, max_success_rate)
    print(max_success_rate)
    success_count = 0

eps, step, total_steps, action[0], reward, loss_a, loss_c, agent_mind_ddpg.epsilon, acc_reward/step
  0 ;   49;     49; +0.17; -15.00; +0.00; +0.00; 1.000; 1.64
0
  1 ;   16;     65; +0.39; -1.01; +0.00; +0.00; 1.000; -0.40

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([64, 64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


  1 ;  150;    199; -1.00; -0.76; +1.48; +19.20; 0.999; -2.20

KeyboardInterrupt: 

In [ ]:
visualize(agent_mind_ddpg, total_eps=8, map_path=terrain, gif_path=gif_path, gif_name="DDPG_"+str(eps).zfill(4)+".gif")

# SAC

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
import sac

batch_size = 64
LOG_SIG_MAX = 2
LOG_SIG_MIN = -20
epsilon = 1e-6


class PolicyNetGaussian(nn.Module):
    def __init__(self):
        super(PolicyNetGaussian, self).__init__()
        self.layer1 = nn.Linear(23, 512)
        self.layer2 = nn.Linear(512, 512)
        self.layer3 = nn.Linear(512, 512)
        self.layer_4_mean = nn.Linear(512, 2)
        self.layer_4_standard_log = nn.Linear(512, 2)

    def forward(self, s):
        hidden_layer_1 = F.relu(self.layer1(s))
        hidden_layer_2 = F.relu(self.layer2(hidden_layer_1))
        hidden_layer_3 = F.relu(self.layer3(hidden_layer_2))
        return self.layer_4_mean(hidden_layer_3), torch.clamp(self.layer_4_standard_log(hidden_layer_3), min=LOG_SIG_MIN, max=LOG_SIG_MAX)
    
    def sample(self, s):
        a_mean, standard_log = self.forward(s)
        a_std = standard_log.exp()
        flow = Normal(a_mean, a_std)
        position_x = flow.rsample()
        A_ = torch.tanh(position_x)
        log_prob = flow.log_prob(position_x) -  torch.log(1 - A_.pow(2) + epsilon)
        return A_, log_prob.sum(1, keepdim=True), torch.tanh(a_mean)

agent_mind_sac = sac.SAC( model = [PolicyNetGaussian, QNet], b_size = batch_size)

In [ ]:

if load_model:
    print("Load model ...", model_path)
    agent_mind_sac.save_load_model("load", model_path)

mother_nature = IndoorDeepRL(map_path=terrain)
total_step = 0
max_success_rate = 0
success_count = 0
print("eps, step, total_step, action[0], reward, loss_a, loss_c, agent_mind_sac.alpha, acc_reward/step")

for eps in range(4500):
    state = mother_nature.createInstance()
    step = 0
    loss_a = loss_c = 0.
    acc_reward = 0.

    while True:
        if training:
            action = agent_mind_sac.choose_action(state, eval=False)
        else:
            action = agent_mind_sac.choose_action(state, eval=True)
        
        state_next, reward, terminated = mother_nature.step(action)
        end = 0 if terminated else 1
        agent_mind_sac.store_transition(state, action, reward, state_next, end)

        displayed = mother_nature.render(gui=render)

        loss_a = loss_c = 0.
        if total_step > batch_size and training:
            loss_a, loss_c = agent_mind_sac.learn()

        step += 1
        total_step += 1
        acc_reward += reward

        print('\r {:3d}; {:4d}; {:6d}; {:+.2f}; {:+.2f}; {:+.2f}; {:+.2f};  {:.3f}; {:.2f}  '\
                .format(eps, step, total_step, action[0], reward, loss_a, loss_c, agent_mind_sac.alpha, acc_reward/step), end='')

        state = state_next.copy()
        if terminated or step>200:
            if reward > 5:
                success_count += 1
            print()
            break

    max_success_rate = performance_measure(eps, agent_mind_sac, success_count, max_success_rate)
    success_count = 0

In [ ]:
visualize(agent_mind_sac, total_eps=8, map_path=terrain, gif_path=gif_path, gif_name="SAC_"+str(eps).zfill(4)+".gif")

# PPO Agent training

In [ ]:
import ppo


class PPOPolicy(nn.Module):
    def __init__(self):
        super(PPOPolicy, self).__init__()
        self.layer1 = nn.Linear(23, 512)
        self.layer2 = nn.Linear(512, 512)
        self.layer3 = nn.Linear(512, 512)
        self.layer_4_mean = nn.Linear(512, 2)
        self.layer_4_standard_log = nn.Linear(512, 2)

    def forward(self, s):
        hidden_layer_1 = F.relu(self.layer1(s))
        hidden_layer_2 = F.relu(self.layer2(hidden_layer_1))
        hidden_layer_3 = F.relu(self.layer3(hidden_layer_2))
        return  torch.tanh(self.layer_4_mean(hidden_layer_3)), self.layer_4_standard_log(hidden_layer_3)
    
    def distribution(self, s):
        a_mean, standard_log = self.forward(s)
        return Normal(a_mean, standard_log.exp())

    def sample(self, s):
        flow = self.distribution(s)
        a_samp = flow.sample()
        return a_samp, flow.log_prob(a_samp)

class ValueNet(nn.Module):
    def __init__(self):
        super(ValueNet, self).__init__()
        self.layer1 = nn.Linear(23, 512)
        self.layer2 = nn.Linear(512, 512)
        self.layer3 = nn.Linear(512, 512)
        self.layer4 = nn.Linear(512, 1)
    
    def forward(self, s):
        hidden_layer_1 = F.relu(self.layer1(s))
        hidden_layer_2 = F.relu(self.layer2(hidden_layer_1))
        hidden_layer_3 = F.relu(self.layer3(hidden_layer_2))
        return self.layer4(hidden_layer_3)

agent_mind_ppo = ppo.PPO(model = [PPOPolicy, ValueNet], batch_size = 1)

In [ ]:
from torch.distributions import Normal

render = True
if load_model:
    print("Load model ...", model_path)
    agent_mind_ppo.save_load_model("load", model_path)


mother_nature = IndoorDeepRL(map_path=terrain)
total_step = 0
max_success_rate = 0
success_count = 0
print("eps, step, total_step, action[0], reward, acc_reward/step")

for eps in range(4500):
    state = mother_nature.createInstance()
    step = 0
    loss_a = loss_c = 0.
    acc_reward = 0.

    while True:
        if training:
            action, policy_log = agent_mind_ppo.choose_action(state, eval=False)
        else:
            action, policy_log = agent_mind_ppo.choose_action(state, eval=True)
        
        state_next, reward, terminated = mother_nature.step(action)

        if terminated:
            end = 0
        else:
            end = 1

        agent_mind_ppo.store_transition(state, action, reward, state_next, end, policy_log)
        displayed = mother_nature.render(gui=render)

        step += 1
        total_step += 1

        acc_reward += reward
        print('\r {:3d}; {:4d}; {:6d}; {:+.2f}; {:+.2f}; {:.2f} '\
                .format(eps, step, total_step, action[0], reward, acc_reward/step), end='')

        state = state_next.copy()
        if terminated or step>200:
            if reward > 5:
                success_count += 1
            print()
            break

    agent_mind_ppo.learn()

    max_success_rate = performance_measure(eps, agent_mind_ddpg, success_count, max_success_rate)
    success_count = 0

In [ ]:
visualize(agent_mind_ppo, total_eps=4, map_path=terrain, gif_path=gif_path, gif_name="PPO_"+str(eps).zfill(4)+".gif")